Consider the manufacturing of
pistons where each piston consists of a rod and a tube that need to be assembled together to form the piston. Each rod consists of the main body and a special kit that is welded to the rod (the kit needs to be collected from warehouse and then assembled). The rod body is sawn from a large metal stick. The tube can also be sawn from a larger tube. Both rod body and tube must be collected together from the warehouse to ensure that their diameters fit. If the tube is not available, it can be bought from an external supplier. In any case some welding is necessary to be done on the tube before it can be assembled with the rod. Finally, between
sawing and welding, both rod and tube must be cleared of metal cuts produced by sawing. Assume that welding and
sawing operations require ten time units, assembly operation requires five time units, clearing can be done in two time units, and the material is collected from
warehouse in one time unit. If the tube is bought from an external supplier then it takes fifty time units to get it. Moreover, tube and rod must cool-down after welding which takes five time units. We assume that shipPiston MaxTime = 70.

source: Barták, Roman, and Ondrej Cepek. "Temporal Networks with Alternatives: Complexity and Model." FLAIRS Conference. 2007.

![](../pics/2023-08-06-14-11-22.png)

In [2]:
from docplex.cp.model import CpoModel
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


import matplotlib.pyplot as plt
%matplotlib inline
#Change the plot size
from pylab import rcParams
rcParams['figure.figsize'] = 35, 10
import docplex.cp.utils_visu as visu

In [44]:
Required_jobs = [['CollectKit',1, 'collect special kit from warehouse'],
                 ['AssembleKit', 5, 'assemble special kit' ],
                ['WeldRod',10 + 5, 'weld special kit to the rod, and cool-down'], 
                ['CollectMaterial',1, 'collect material for rod body'],
                ['SawRod',10+ 5, 'saw the rod body from a large metal stick'],
                ['WeldTube',10, 'necessary welding of tube, and cool-down'],  
                ['ClearRod', 2 ,  'rod must be cleared of metal cuts produced by sawing'  ] ,
                ['AssemblePiston',5 ,  'Assemble rod and tube'  ]]





Optional_jobs = [['SawTube',10, 'saw tube from a larger tube' ],
                 ['BuyTube',50, 'buy tube from an external supplier' ],
                ['ClearTube', 2 ,  'Tube must be cleared of metal cuts produced by sawing'  ] 
               ]
                 
                 
    
Required_jobs = pd.DataFrame( Required_jobs,  columns = ['job', 'duration', 'comments'])
Optional_jobs = pd.DataFrame( Optional_jobs,  columns = ['job', 'duration', 'comments'])

display(Required_jobs)
display(Optional_jobs)

Precedence = pd.DataFrame([['CollectKit', 'AssembleKit'],
                          ['AssembleKit', 'WeldRod'],
                          ['CollectMaterial','SawRod'],
                          ['SawRod','ClearRod'],
                          ['ClearRod','WeldRod'],
                          ['SawTube','ClearTube'],
                          ['AssembleKit', 'WeldRod'],
                          ['WeldTube', 'AssemblePiston'],
                          ['WeldRod',  'AssemblePiston'],
                          ['AssemblePiston', 'ShipPiston']],
                          columns = ["beforeTask", "afterTask"])

display(Precedence)

#Simultaneous_jobs= pd.DataFrame([

,job,duration,comments
0,CollectKit,1,collect special kit from warehouse
1,AssembleKit,5,assemble special kit
2,WeldRod,15,"weld special kit to the rod, and cool-down"
3,CollectMaterial,1,collect material for rod body
4,SawRod,15,saw the rod body from a large metal stick
5,WeldTube,10,"necessary welding of tube, and cool-down"
6,ClearRod,2,rod must be cleared of metal cuts produced by sawing
7,AssemblePiston,5,Assemble rod and tube


,job,duration,comments
0,SawTube,10,saw tube from a larger tube
1,BuyTube,50,buy tube from an external supplier
2,ClearTube,2,Tube must be cleared of metal cuts produced by sawing


,beforeTask,afterTask
0,CollectKit,AssembleKit
1,AssembleKit,WeldRod
2,CollectMaterial,SawRod
3,SawRod,ClearRod
4,ClearRod,WeldRod
5,SawTube,ClearTube
6,AssembleKit,WeldRod
7,WeldTube,AssemblePiston
8,WeldRod,AssemblePiston
9,AssemblePiston,ShipPiston


In [51]:
mdl = CpoModel()

Required_jobs['Required_DV']=0
for index, row in Required_jobs.iterrows():
    dv =  mdl.interval_var(size=row.duration, name="{}".format(row.job))
    Required_jobs.at[index, 'Required_DV']=dv

display(Required_jobs.head())


Optional_jobs['Optional_DV']=0
for index, row in Optional_jobs.iterrows():
    dv =  mdl.interval_var( optional = True , size=row.duration, name="{}".format(row.job))
    Optional_jobs.at[index, 'Optional_DV']=dv


display(Optional_jobs.head())

ShipPiston = mdl.interval_var(  end = (0,70), name='ShipPiston')

,job,duration,comments,Required_DV
0,CollectKit,1,collect special kit from warehouse,CollectKit = intervalVar(size=1)
1,AssembleKit,5,assemble special kit,AssembleKit = intervalVar(size=5)
2,WeldRod,15,"weld special kit to the rod, and cool-down",WeldRod = intervalVar(size=15)
3,CollectMaterial,1,collect material for rod body,CollectMaterial = intervalVar(size=1)
4,SawRod,15,saw the rod body from a large metal stick,SawRod = intervalVar(size=15)


,job,duration,comments,Optional_DV
0,SawTube,10,saw tube from a larger tube,"SawTube = intervalVar(optional, size=10)"
1,BuyTube,50,buy tube from an external supplier,"BuyTube = intervalVar(optional, size=50)"
2,ClearTube,2,Tube must be cleared of metal cuts produced by sawing,"ClearTube = intervalVar(optional, size=2)"


In [52]:
DV_dict= dict(zip(Required_jobs['job'], Required_jobs['Required_DV']))
d2= dict(zip(Optional_jobs['job'], Optional_jobs['Optional_DV']))
DV_dict.update(d2)
DV_dict['ShipPiston'] = ShipPiston
DV_dict

{'CollectKit': <docplex.cp.expression.CpoIntervalVar at 0x203387b38b0>,
 'AssembleKit': <docplex.cp.expression.CpoIntervalVar at 0x203387b3ca0>,
 'WeldRod': <docplex.cp.expression.CpoIntervalVar at 0x203387b3680>,
 'CollectMaterial': <docplex.cp.expression.CpoIntervalVar at 0x203387b2340>,
 'SawRod': <docplex.cp.expression.CpoIntervalVar at 0x203387b37d0>,
 'WeldTube': <docplex.cp.expression.CpoIntervalVar at 0x203387b3530>,
 'ClearRod': <docplex.cp.expression.CpoIntervalVar at 0x203387b3610>,
 'AssemblePiston': <docplex.cp.expression.CpoIntervalVar at 0x203387e82e0>,
 'SawTube': <docplex.cp.expression.CpoIntervalVar at 0x203387b36f0>,
 'BuyTube': <docplex.cp.expression.CpoIntervalVar at 0x203387e83c0>,
 'ClearTube': <docplex.cp.expression.CpoIntervalVar at 0x203387e8580>,
 'ShipPiston': <docplex.cp.expression.CpoIntervalVar at 0x203387b3b50>}

In [53]:
# Temporal constraints.TASK_PRECEDENCES  

Precedence ["beforeTaskDV"] = Precedence ["beforeTask"].apply(lambda x:DV_dict[x])
Precedence ["afterTaskDV"] = Precedence ["afterTask"].apply(lambda x:DV_dict[x])

display(Precedence)

for index, row in Precedence.iterrows():
    mdl.add( mdl.end_before_start(row.beforeTaskDV, row.afterTaskDV))

,beforeTask,afterTask,beforeTaskDV,afterTaskDV
0,CollectKit,AssembleKit,CollectKit = intervalVar(size=1),AssembleKit = intervalVar(size=5)
1,AssembleKit,WeldRod,AssembleKit = intervalVar(size=5),WeldRod = intervalVar(size=15)
2,CollectMaterial,SawRod,CollectMaterial = intervalVar(size=1),SawRod = intervalVar(size=15)
3,SawRod,ClearRod,SawRod = intervalVar(size=15),ClearRod = intervalVar(size=2)
4,ClearRod,WeldRod,ClearRod = intervalVar(size=2),WeldRod = intervalVar(size=15)
5,SawTube,ClearTube,"SawTube = intervalVar(optional, size=10)","ClearTube = intervalVar(optional, size=2)"
6,AssembleKit,WeldRod,AssembleKit = intervalVar(size=5),WeldRod = intervalVar(size=15)
7,WeldTube,AssemblePiston,WeldTube = intervalVar(size=10),AssemblePiston = intervalVar(size=5)
8,WeldRod,AssemblePiston,WeldRod = intervalVar(size=15),AssemblePiston = intervalVar(size=5)
9,AssemblePiston,ShipPiston,AssemblePiston = intervalVar(size=5),ShipPiston = intervalVar(end=0..70)


In [54]:
print("\nSolving model....")
msol = mdl.solve(TimeLimit=120,  trace_log=False)

# Print solution
print("Solve status: " + msol.get_solve_status())

msol.print_solution()



Solving model....
Solve status: Feasible
-------------------------------------------------------------------------------
Model constraints: 10, variables: integer: 0, interval: 11, sequence: 0
Solve status: Feasible
Search status: SearchCompleted, stop cause: SearchHasNotBeenStopped
Solve time: 0.01 sec
-------------------------------------------------------------------------------
Variables:
   AssembleKit = IntervalVarValue(start=1, end=6, size=5)
   AssemblePiston = IntervalVarValue(start=33, end=38, size=5)
   ClearRod = IntervalVarValue(start=16, end=18, size=2)
   ClearTube = ()
   CollectKit = IntervalVarValue(start=0, end=1, size=1)
   CollectMaterial = IntervalVarValue(start=0, end=1, size=1)
   SawRod = IntervalVarValue(start=1, end=16, size=15)
   SawTube = ()
   ShipPiston = IntervalVarValue(start=38, end=38, size=0)
   WeldRod = IntervalVarValue(start=18, end=33, size=15)
   WeldTube = IntervalVarValue(start=0, end=10, size=10)


In [23]:
mdl.write_information()

Model: 423125861
 - source file: C:/Users/m_gor/AppData/Local/Temp/ipykernel_62204/423125861.py
 - modeling time: 0.99 sec
 - number of integer variables:  0
 - number of interval variables: 8
 - number of sequence variables: 0
 - number of state functions:    0
 - number of float variables:    0
 - number of constraints:        8
 - number of root expressions:   8
 - number of expression nodes:   16
 - operations:                   endBeforeStart: 8


In [3]:
import os
import subprocess
CPLEX_path  = "C:/IBM/ILOG/CPLEX_Studio2211/opl/bin/x64_win64"
os.chdir(CPLEX_path)

# The path of the project that containts all the .project, OPL. and etc. files

path = "C:/Users/m_gor/Desktop/OptimizationDemos/CP/JobShop/"
mod_file =path+ "fact.mod"

command = "oplrun  %s " %(mod_file )
print(command)

output = subprocess.getoutput(command)
print(output)

oplrun  C:/Users/m_gor/Desktop/OptimizationDemos/CP/JobShop/fact.mod 

<<< setup


<<< generate

 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Satisfiability problem - 14 variables, 14 constraints
 ! Initial process time : 0.01s (0.01s extraction + 0.00s propagation)
 !  . Log search space  : 51.4 (before), 51.4 (after)
 !  . Memory usage      : 303.5 kB (before), 303.5 kB (after)
 ! Using parallel search with 8 workers.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed    W       Branch decision
 ! Using iterative diving.
 *                     11  0.02s        1            -
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 96
 ! Number of fails        : 4
 ! Total memory usage     : 3.0 MB (2.6 MB CP Optimi

In [4]:
from docplex.cp.expression import transition_matrix

In [43]:
NbHouses = 5

WorkerNames = ["Joe", "Jim"]

TaskNames = ["masonry", "carpentry", "plumbing", 
             "ceiling", "roofing", "painting", 
             "windows", "facade", "garden", "moving"]

Duration =  [35, 15, 40, 15, 5, 10, 5, 10, 5, 5]

Worker = {"masonry"  : "Joe" , 
          "carpentry": "Joe" , 
          "plumbing" : "Jim" , 
          "ceiling"  : "Jim" , 
          "roofing"  : "Joe" , 
          "painting" : "Jim" , 
          "windows"  : "Jim" , 
          "facade"   : "Joe" , 
          "garden"   : "Joe" , 
          "moving"   : "Jim"}

ReleaseDate = [  0,     0,   151,    59,   243]
DueDate     = [120,   212,   304,   181,   425]
Weight      = [100.0, 100.0, 100.0, 200.0, 100.0]

Precedences = [("masonry", "carpentry"),("masonry", "plumbing"),
               ("masonry", "ceiling"), ("carpentry", "roofing"),
               ("ceiling", "painting"), ("roofing", "windows"),  
               ("roofing", "facade"), ("plumbing", "facade"),
               ("roofing", "garden"), ("plumbing", "garden"),
               ("windows", "moving"), ("facade", "moving"),  
               ("garden", "moving"), ("painting", "moving")]

Houses = range(NbHouses)

mdl2 = CpoModel()
houses = [mdl2.interval_var(start=(ReleaseDate[i], INTERVAL_MAX), name="house"+str(i)) for i in Houses]

TaskNames_ids = {}
itvs = {}
for h in Houses:
    for i,t in enumerate(TaskNames):
        _name = str(h)+"_"+str(t)
        itvs[(h,t)] = mdl2.interval_var(size=Duration[i], name=_name)
        TaskNames_ids[_name] = i


for h in Houses:
    for p in Precedences:
        mdl2.add(mdl2.end_before_start(itvs[(h,p[0])], itvs[(h,p[1])]) )

for h in Houses:
    mdl2.add( mdl2.span(houses[h], [itvs[(h,t)] for t in TaskNames] ) )


transitionTimes = transition_matrix([[int(abs(i - j)) for j in Houses] for i in Houses])

workers = {w : mdl2.sequence_var([ itvs[(h,t)] for h in Houses for t in TaskNames if Worker[t]==w ], 
                                types=[h for h in Houses for t in TaskNames if Worker[t]==w ], name="workers_"+w)   
           for w in WorkerNames}

for w in WorkerNames:
    mdl2.add( mdl2.no_overlap(workers[w], transitionTimes) )

# create the obj and add it.
mdl2.add( 
    mdl2.minimize( 
        mdl2.sum(Weight[h] * mdl2.max([0, mdl2.end_of(houses[h])-DueDate[h]]) + mdl2.length_of(houses[h]) for h in Houses) 
    ) 
)
# Solve the model
print("\nSolving model....")
msol2 = mdl2.solve(FailLimit=30000, trace_log=False)
print("done")

if msol2:
    print("Cost will be " + str(msol2.get_objective_values()[0]))
else:
    print("No solution found")



Solving model....
done
Cost will be 17065


In [40]:
mdl2.print_information()

Model: 497071845
 - source file: C:/Users/m_gor/AppData/Local/Temp/ipykernel_61324/497071845.py
 - modeling time: 0.01 sec
 - number of integer variables:  0
 - number of interval variables: 55
 - number of sequence variables: 2
 - number of state functions:    0
 - number of float variables:    0
 - number of constraints:        78
 - number of root expressions:   78
 - number of expression nodes:   188
 - operations:                   endBeforeStart: 70, endOf: 5, lengthOf: 5, max: 5, minimize: 1, minus: 5, noOverlap: 2, plus: 5, span: 5, sum: 1, times: 5
